<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Password Cracker analysis and idea testing

In [293]:
import pandas as pd
import crypt
import hashlib

df = pd.read_csv('shadow', sep=':',header=None, usecols=[0,1], names=['username','password'])
df = df.set_index('username')

In [294]:
df

,password
username,
root,$6$QlJt0cnr$hmgN/fzUrHFFI1SaGXVNzE060TPuwsZdzP...
daemon,*
bin,*
sys,*
sync,*
games,*
man,*
lp,*
mail,*


In [295]:
df = df[(df.password != '!') & (df.password != '*') & (df.password != 'x')] 

In [296]:
df

,password
username,
root,$6$QlJt0cnr$hmgN/fzUrHFFI1SaGXVNzE060TPuwsZdzP...
backup,$6$Tye3KuC5$rVIT3u5M9IhZZI.jRanteGT3o7DbkLFWb/...
allison,$6$sPsSvR2J$wk59pi4or6QR5IobArTZpn4k7i2jZQ07pY...
paul,$6$YGG4oFLp$avrVGY6.S59aApmCY/60A7AWfGDBh/zI7L...
dr_balustrade,$6$3kgge6ym$OcIOZS8bJy41YsLYXToOW2Ag3imG1KEXkP...


In [297]:

passwdtocrack = df.password['dr_balustrade']
passwdtocrack

'$6$3kgge6ym$OcIOZS8bJy41YsLYXToOW2Ag3imG1KEXkPgQpnbSfCBIYE26Kp42QHGeAyV3L4zPsa/AAuAsLXx9QCXtyF/xX0'

In [298]:
'dr_balustrade' in df.index

True

In [299]:
crypttype = passwdtocrack.split('$')[1]
crypttype

'6'

In [300]:
salt = passwdtocrack.split("$")[2]
salt

'3kgge6ym'

In [301]:
hashedpwd = passwdtocrack.split("$")[3]
hashedpwd

'OcIOZS8bJy41YsLYXToOW2Ag3imG1KEXkPgQpnbSfCBIYE26Kp42QHGeAyV3L4zPsa/AAuAsLXx9QCXtyF/xX0'

In [302]:
insalt = '$' + crypttype + '$' + salt + '$'
insalt

'$6$3kgge6ym$'

In [303]:
wordundertest = 'pinky'
crypt.crypt('pinky',salt=salt)

'3kE9YicCkiyT6'

In [304]:
passwdtocrack == crypt.crypt('pinky', passwdtocrack)

False

In [305]:
from passlib.hash import sha512_crypt
from passlib.hosts import linux_context

sha512_crypt.hash('pinky', rounds=5000, salt=salt)

'$6$3kgge6ym$OcIOZS8bJy41YsLYXToOW2Ag3imG1KEXkPgQpnbSfCBIYE26Kp42QHGeAyV3L4zPsa/AAuAsLXx9QCXtyF/xX0'

In [306]:
linux_context.verify('pinky', passwdtocrack)

True

In [307]:
passwdtocrack == sha512_crypt.hash('pinky', rounds=5000, salt=salt)

True

In [308]:
dict=open('dicttest.txt','r', encoding="ascii")
for word in dict:
   # word=word.strip()
    word=word.strip('\n')
    print(word)
    print(linux_context.verify(word, passwdtocrack))
    print(sha512_crypt.hash(str(word), rounds=5000, salt=salt))
    

pinky
True
$6$3kgge6ym$OcIOZS8bJy41YsLYXToOW2Ag3imG1KEXkPgQpnbSfCBIYE26Kp42QHGeAyV3L4zPsa/AAuAsLXx9QCXtyF/xX0
test
False
$6$3kgge6ym$hEp9egWLZ6M3eAc3NI4giq6VmFIWomGuVxUed.LclFy.8CyhWoJTqm9Wiuj3D.7.eH55IbWi10Q1dSU0QyG4o1
hello
False
$6$3kgge6ym$LytWHGgQ5PUSKqQyLErs5pedw/a2bF6PDgknDJSQMeuyXgNKbgLCqUCGEI889XSLQbHQSem64zKDtadU0GDeA1
password123
False
$6$3kgge6ym$nOTC.eiZDYYm1hbpSB559OGgF9mRrj72T3uIdDqF15AGTdIsMUMbgIEWNQ51xepO4Yhxsg32BREfAxuFNLaEn0


In [309]:
for index, row in df.iterrows():
    print(index)

root
backup
allison
paul
dr_balustrade
